In [41]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import time
from bs4 import BeautifulSoup

# Specify the path to Chromedriver (replace with your actual path)
chromedriver_path = 'C:/Users/domen/Downloads/chromedriver_win32/chromedriver.exe'

# Create Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Optional: Run headless (without opening a browser window)

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# URL pattern for the pages
url_pattern = 'https://www.capfriendly.com/browse/active?stats-season=2023&display=signing-team,draft,waivers-exempt,signing-status,expiry-year,performance-bonus,signing-bonus,caphit-percent,aav,length,minors-salary,base-salary,arbitration-eligible,type,signing-age,signing-date,arbitration,extension&hide=age,position,handed&pg={}'

# Initialize an empty DataFrame to store the data
all_data = pd.DataFrame()

# Loop through all pages (31 pages)
for page_number in range(1, 32):
    # Load the page in the WebDriver
    driver.get(url_pattern.format(page_number))

    # Wait for the page to load (you may need to adjust the sleep time)
    import time
    time.sleep(5)  # Wait for 5 seconds to allow the page to load

    # Get the HTML content of the page
    page_source = driver.page_source

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find the table containing the data
    table = soup.find('table')

    # Read the table data into a DataFrame
    data = pd.read_html(str(table))[0]

    # Concatenate the data to the main DataFrame
    all_data = pd.concat([all_data, data], ignore_index=True)

# Close the WebDriver
driver.quit()

# Create copy of data
cap_data = all_data.copy()

# Split the 'DRAFTED' column into four new columns using regular expressions
draft_split = cap_data['DRAFTED'].str.extract(r'(\d+) - Round (\d+) - (\d{4}) \(([^)]+)\)')
draft_split.columns = ['draft_round', 'draft_pick', 'draft_year', 'draft_team']

# Add the new columns to your DataFrame
cap_data = pd.concat([cap_data, draft_split], axis=1)

# Fix the player name
cap_data['player_full_name'] = cap_data['PLAYER'].str.split('.').str[1:].str.join('').str.strip()

# Function to attempt parsing in different date formats
def parse_date(date_str):
    formats = ["%b. %d, %Y", "%b %d, %Y", "%b. %d, %Y", "%b %d, %Y"]
    for date_format in formats:
        try:
            return pd.to_datetime(date_str, format=date_format).strftime("%b. %d, %Y")  # Format as string
        except ValueError:
            pass
    return None

# Apply date parsing to "SIGNING DATE" column
cap_data['SIGNING YEAR'] = cap_data['SIGNING DATE'].apply(parse_date)
cap_data['EXP. YEAR'] = pd.to_datetime(cap_data['EXP. YEAR'], format='%Y')
cap_data['YEARS LEFT'] = (cap_data['EXP. YEAR'] - pd.to_datetime(cap_data['SIGNING DATE'], format="%b. %d, %Y")).dt.days // 365

# Save the scraped data to a CSV file
cap_data.to_csv("capfriendly_data.csv", index=False)  # Specify the desired filename

<ipython-input-41-b01fa59e0702>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
<ipython-input-41-b01fa59e0702>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
<ipython-input-41-b01fa59e0702>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
<ipython-input-41-b01fa59e0702>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
<ipython-input-41-b01fa59e0702>:

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import time
from bs4 import BeautifulSoup

# Specify the path to Chromedriver (replace with your actual path)
chromedriver_path = 'C:/Users/domen/Downloads/chromedriver_win32/chromedriver.exe'

# Create Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Optional: Run headless (without opening a browser window)

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# URL pattern for the pages
url_pattern = 'https://www.capfriendly.com/browse/active?stats-season=2023&display=signing-team,draft,waivers-exempt,signing-status,expiry-year,performance-bonus,signing-bonus,caphit-percent,aav,length,minors-salary,base-salary,arbitration-eligible,type,signing-age,signing-date,arbitration,extension&hide=age,position,handed&pg={}'

# Initialize an empty DataFrame to store the data
all_data = pd.DataFrame()

# Loop through all pages (31 pages)
for page_number in range(1, 32):
    # Load the page in the WebDriver
    driver.get(url_pattern.format(page_number))

    # Wait for the page to load (you may need to adjust the sleep time)
    import time
    time.sleep(5)  # Wait for 5 seconds to allow the page to load

    # Get the HTML content of the page
    page_source = driver.page_source

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find the table containing the data
    table = soup.find('table')

    # Read the table data into a DataFrame
    data = pd.read_html(str(table))[0]

    # Concatenate the data to the main DataFrame
    all_data = pd.concat([all_data, data], ignore_index=True)

# Close the WebDriver
driver.quit()

# Create copy of data
cap_data = all_data.copy()

# Split the 'DRAFTED' column into four new columns using regular expressions
draft_split = cap_data['DRAFTED'].str.extract(r'(\d+) - Round (\d+) - (\d{4}) \(([^)]+)\)')
draft_split.columns = ['DRAFT_ROUND', 'DRAFT_PICK', 'DRAFT_YEAR', 'DRAFT_TEAM']

# Add the new columns to your DataFrame
cap_data = pd.concat([cap_data, draft_split], axis=1)

# Fix the player name
cap_data['PLAYER_FULL_NAME'] = cap_data['PLAYER'].str.split('.').str[1:].str.join('').str.strip()

# Function to attempt parsing in different date formats
def parse_date(date_str):
    formats = ["%b. %d, %Y", "%b %d, %Y", "%b. %d, %Y", "%b %d, %Y"]
    for date_format in formats:
        try:
            return pd.to_datetime(date_str, format=date_format).strftime("%b. %d, %Y")  # Format as string
        except ValueError:
            pass
    return None

# Function to format values
def format_value(value):
    # Remove '$' and ',' and convert to float
    value_float = float(value.replace('$', '').replace(',', ''))
    if value_float >= 1000000:
        return f"${value_float / 1000000:.2f}M"
    else:
        return f"${value_float / 1000:.2f}K"


# Apply date parsing to "SIGNING DATE" column
cap_data['SIGNING_YEAR'] = cap_data['SIGNING DATE'].apply(parse_date)
cap_data['EXPIRY_YEAR'] = pd.to_datetime(cap_data['EXP. YEAR'], format='%Y')
cap_data['YEARS_LEFT'] = (cap_data['EXPIRY_YEAR'] - pd.to_datetime(cap_data['SIGNING_YEAR'], format="%b. %d, %Y")).dt.days // 365
cap_data['CAP_HIT2'] = cap_data['CAP HIT'].apply(format_value)
cap_data['AAV_2'] = cap_data['AAV'].apply(format_value)

# Save the scraped data to a CSV file
cap_data.to_csv("capfriendly_data.csv", index=False)  # Specify the desired filename

c:\Users\domen\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
<ipython-input-1-ef0af0a83783>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
<ipython-input-1-ef0af0a83783>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
<ipython-input-1-ef0af0a83783>:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.615
  (Session info: headless chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF6D35F7892+54818]
	(No symbol) [0x00007FF6D3566AC2]
	(No symbol) [0x00007FF6D341DA3B]
	(No symbol) [0x00007FF6D340C06A]
	(No symbol) [0x00007FF6D340BFB3]
	(No symbol) [0x00007FF6D340A82E]
	(No symbol) [0x00007FF6D340B2EE]
	(No symbol) [0x00007FF6D341775B]
	(No symbol) [0x00007FF6D34284D1]
	(No symbol) [0x00007FF6D342C80A]
	(No symbol) [0x00007FF6D340B937]
	(No symbol) [0x00007FF6D34283AA]
	(No symbol) [0x00007FF6D3497609]
	(No symbol) [0x00007FF6D347E883]
	(No symbol) [0x00007FF6D3453691]
	(No symbol) [0x00007FF6D34548D4]
	GetHandleVerifier [0x00007FF6D395B992+3610402]
	GetHandleVerifier [0x00007FF6D39B1860+3962352]
	GetHandleVerifier [0x00007FF6D39A9D4F+3930847]
	GetHandleVerifier [0x00007FF6D3693646+693206]
	(No symbol) [0x00007FF6D3571628]
	(No symbol) [0x00007FF6D356D934]
	(No symbol) [0x00007FF6D356DA62]
	(No

In [9]:
cap_data = pd.read_csv("capfriendly_data.csv")

# Function to format values
def format_value(value):
    # Remove '$' and ',' and convert to float
    value_float = float(value.replace('$', '').replace(',', ''))
    if value_float >= 1000000:
        return f"${value_float / 1000000:.1f}M"
    else:
        return f"${value_float / 1000:.1f}K"
    
cap_data['CAP_HIT2'] = cap_data['CAP HIT'].apply(format_value)
cap_data['AAV2'] = cap_data['AAV'].apply(format_value)

# Save the scraped data to a CSV file
cap_data.to_csv("capfriendly_data2.csv", index=False)  # Specify the desired filename